# INTRODUCTION

This notebook explores the potential of the Epigraphic Database Heidelberg web API , [EDH API](https://edh-www.adw.uni-heidelberg.de/data/api) in combination with sciencedata.dk as a datastorage (see more about our current progress in using sciencedata.dk [here](https://docs.google.com/document/d/1sojHsxkcAbZH9DpWFuHDomQwTZHPQv_WaAxO_erP6FE/edit?usp=sharing)).

The ambition here is to use cloud based solutions as much as possible, without any dependence on local machines. At the same time, we do not like to rely completely upon google services. 

In [1]:
### REQUIREMENTS
import numpy as np
import math
import pandas as pd

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from urllib.request import urlopen 
from urllib.parse import quote  
from bs4 import BeautifulSoup

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass

import datetime as dt
# for simple paralel computing:
from concurrent.futures import ThreadPoolExecutor
### google drive
#import gspread
#from gspread_dataframe import get_as_dataframe, set_with_dataframe

import sddk

In [47]:
concurrent.futures.__version__

NameError: name 'concurrent' is not defined

## configure session and url

In [2]:
### configure session and url
### in the case of "SDAM_root", the group owner is Vojtech with username 648597@au.dk
conf = sddk.configure_session_and_url("SDAM_root", "6478597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


# EDH via API

The basis form of an request is as follows:
```
https://edh-www.adw.uni-heidelberg.de/data/api/inscriptions/search?
```
With this, to create query based on inscription number, you have to specify the paramenter **hd_nr**, like here:

```
https://edh-www.adw.uni-heidelberg.de/data/api/inscriptions/search?hd_nr=1
```
 (Feel free to explore this in web browser).

Here we use the function ```requests.get()``` to make our requests from python.

## One inscription query example

In [49]:
%%time
inscription_number = 100
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


URL_form = "https://edh-www.adw.uni-heidelberg.de/data/api/inscriptions/search?"

response = requests.get(URL_form + "hd_nr=" + str(inscription_number), headers=headers)
#response
json_data = response.json()
print(json_data)

{'items': [{'work_status': 'provisional', 'type_of_monument': 'stele', 'id': 'HD000100', 'findspot_ancient': 'Uxama', 'edh_geography_uri': 'https://edh-www.adw.uni-heidelberg.de/edh/geographie/9371', 'uri': 'https://edh-www.adw.uni-heidelberg.de/edh/inschrift/HD000100', 'province_label': 'Hispania citerior', 'responsible_individual': 'Gräf', 'trismegistos_uri': 'https://www.trismegistos.org/text/226731', 'findspot_modern': 'El Burgo de Osma', 'diplomatic_text': 'D[ ] / ANELI[ ] / BERVE[ ] / P[ ]IT[', 'language': 'Latin', 'country': 'Spain', 'modern_region': 'Soria', 'literature': 'AE 1983, 0597.; C. García Merino, in: Homenaje al Prof. Martin Almagro Basch 3 (Madrid 1983) 355, Nr. 2; lám. 1, 2. - AE 1983.', 'commentary': ' Text in vier Zeilen, nahezu unlesbar.', 'transcription': 'D[---] / ANELI[---] / BERVE[---] / P[---]IT[------', 'last_update': '2015-05-21'}], 'total': 1, 'limit': '20'}
CPU times: user 29.2 ms, sys: 0 ns, total: 29.2 ms
Wall time: 628 ms


In [0]:
%%time
inscription_number = 100
URL_form = "https://edh-www.adw.uni-heidelberg.de/data/api/inscriptions/search?"

response = requests.get(URL_form + "hd_nr=" + str(inscription_number))
response
json_data = response.json()
print(json_data)

{'total': 1, 'items': [{'province_label': 'Hispania citerior', 'modern_region': 'Soria', 'findspot_ancient': 'Uxama', 'transcription': 'D[---] / ANELI[---] / BERVE[---] / P[---]IT[------', 'commentary': ' Text in vier Zeilen, nahezu unlesbar.', 'id': 'HD000100', 'literature': 'AE 1983, 0597.; C. García Merino, in: Homenaje al Prof. Martin Almagro Basch 3 (Madrid 1983) 355, Nr. 2; lám. 1, 2. - AE 1983.', 'uri': 'https://edh-www.adw.uni-heidelberg.de/edh/inschrift/HD000100', 'language': 'Latin', 'findspot_modern': 'El Burgo de Osma', 'work_status': 'provisional', 'edh_geography_uri': 'https://edh-www.adw.uni-heidelberg.de/edh/geographie/9371', 'last_update': '2015-05-21', 'diplomatic_text': 'D[ ] / ANELI[ ] / BERVE[ ] / P[ ]IT[', 'trismegistos_uri': 'https://www.trismegistos.org/text/226731', 'country': 'Spain', 'responsible_individual': 'Gräf', 'type_of_monument': 'stele'}], 'limit': '20'}
CPU times: user 15.5 ms, sys: 95 µs, total: 15.6 ms
Wall time: 854 ms


In [12]:
%%time 


### the actual data are part of the tag "items"
pd.DataFrame(json_data["items"]) 

CPU times: user 1.05 ms, sys: 11.6 ms, total: 12.7 ms
Wall time: 11.6 ms


,commentary,language,findspot_ancient,last_update,work_status,modern_region,edh_geography_uri,uri,id,diplomatic_text,trismegistos_uri,findspot_modern,province_label,type_of_monument,country,literature,responsible_individual,transcription
0,"Text in vier Zeilen, nahezu unlesbar.",Latin,Uxama,2015-05-21,provisional,Soria,https://edh-www.adw.uni-heidelberg.de/edh/geog...,https://edh-www.adw.uni-heidelberg.de/edh/insc...,HD000100,D[ ] / ANELI[ ] / BERVE[ ] / P[ ]IT[,https://www.trismegistos.org/text/226731,El Burgo de Osma,Hispania citerior,stele,Spain,"AE 1983, 0597.; C. García Merino, in: Homenaje...",Gräf,D[---] / ANELI[---] / BERVE[---] / P[---]IT[--...


# Version 1: Extracting inscriptions one by one (using simple paralel computing)

In [54]:
def get_inscription_data(num):
    try:
        response = requests.get(URL_form + "hd_nr=" + str(num), headers = headers)
        json_data_items = response.json()["items"]
    except:
        time.sleep(1)
        try:
            response = requests.get(URL_form + "hd_nr=" + str(num), headers = headers)
            json_data_items = response.json()["items"]
        except:
            json_data_items = [{}]
    return json_data_items

In [0]:
inscriptions_data_df.head(5)

,people,work_status,findspot_modern,last_update,responsible_individual,width,language,literature,height,diplomatic_text,not_before,depth,material,trismegistos_uri,transcription,commentary,edh_geography_uri,country,uri,province_label,modern_region,type_of_monument,present_location,findspot_ancient,not_after,type_of_inscription,id,letter_size,social_economic_legal_history,findspot,year_of_find,geography,religion,fotos,military,external_image_uris
0,"[{'name': 'L. Ponponius(!) Rufus', 'age: years...",checked with photo,Roma,2014-10-10,Cowey,19 cm,Greek-Latin,"CIG 6916.; AE 1984, 0109. (B); P. Lombardi, Ti...",45 cm,L PONPONIVS RVFVS / VIXIT ANOS XXVII / EIA PON...,0101,5.4 cm,marble: rocks - metamorphic rocks,https://www.trismegistos.org/text/177036,L(ucius) Ponponius(!) Rufus / vixit an(n)os XX...,Wiederverwendung der Tafel als TÃ¼rpfosten. D...,https://edh-www.adw.uni-heidelberg.de/edh/geog...,Italy,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Achaia,Lazio,tabula,"Roma, Mus. Naz. Rom.","Kephallenia, aus",0200,epitaph,HD001917,1-2 cm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'gender': 'male', 'cognomen': 'ÎÎ±Î»Î»ÎµÎ½Ï...",checked with photo,"Patrasso - AthÃ­nai, zwischen",2012-03-15,GrÃ¤f,30 cm,Greek-Latin,"CIL 03, 00572.; CIL 03, 07306.; IG 02 (2. Aufl...",146 cm,[ ]ΥΤΟΚΡΑΤΟΡΙ / [ ]ΑΙΣΑΡΙ / [[[ ]]] / [ ]ΥΣΕΒΕ...,0395,NaN,"Marmor, geÃ¤dert / farbig",https://www.trismegistos.org/text/177037,[Α]ὐτοκράτορι / [Κ]αίσαρι / [[[---]]] / [Ε]ὐσε...,Meilenstein mit zwei griechischen Inschriften...,https://edh-www.adw.uni-heidelberg.de/edh/geog...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Achaia,AttikÃ­,mile-/leaguestone,"AthÃ­nai, Epigr. Mus.","Athenae, bei",0397,mile-/leaguestone,HD002097,2.7 cm,data available,"Dafni, byzantinisches Kloster, bei, sekundÃ¤r ...",NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,no image,AthÃ­nai,2011-04-04,Cowey,(17) cm,Latin,"CIL 03, 06101.; M. Å aÅ¡el Kos, Inscriptiones ...",(15) cm,]S HOSTIVM DEPRESSE[ ] / [ ] CXIIX BELLO MARIT...,-0038,12.5 cm,NaN,https://www.trismegistos.org/text/177038,------ nave]s hostium depresse[rit ---] / [---...,Es handelt sich um ein Elogium fÃ¼r Agrippa. ...,https://edh-www.adw.uni-heidelberg.de/edh/geog...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Achaia,AttikÃ­,base,"AthÃ­nai, Epigr. Mus.",Athenae,-0012,elogium,HD002919,6.5 cm,NaN,"Roma-Augustus Tempel, Akropolis",1866,NaN,NaN,NaN,NaN,NaN
3,"[{'cognomen': 'Traianus Hadrianus', 'gender': ...",checked with photo,AthÃ­nai,2009-11-17,Cowey,76 cm,Greek-Latin,"CIL 03, 00548. (B); CIL 03, 07281.; PIR (2. Au...",112 cm,[ ]MP CAES DIVI TRAIANI PAR / THICI FIL DIVI N...,0132,48 cm,NaN,https://www.trismegistos.org/text/177039,[I]mp(eratori) Caes(ari) divi Traiani Par/thic...,(B): Am Anfang von Z. 2 fehlt das TI von nepoti.,https://edh-www.adw.uni-heidelberg.de/edh/geog...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Achaia,AttikÃ­,statue base,"AthÃ­nai, Epigr. Mus.",Athenae,NaN,honorific inscription,HD002922,NaN,NaN,NaN,NaN,data available,NaN,NaN,NaN,NaN
4,"[{'gender': 'male', 'cognomen': 'Traianus+ Had...",no image,AthÃ­nai,2011-04-04,Cowey,(41) cm,Latin,"CIL 03, 06102.; CIL 03, 07283.; AE 1984, 0822....",(20) cm,[ ] / [ ] / [ ]D[ ] / [ ]R P XVI COS III P P [...,0132,(15) cm,marble: rocks - metamorphic rocks,https://www.trismegistos.org/text/177040,[Imp(eratori) Caesari divi Traiani] / [Parthic...,Rekonstruktion des Inschriftentextes nach CIL...,https://edh-www.adw.uni-heidelberg.de/edh/geog...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Achaia,AttikÃ­,statue base,"AthÃ­nai, Epigr. Mus.",Athenae,NaN,honorific inscription,HD002925,3.8 cm,NaN,"\""Stoa Hadriani\"" (CIL)",NaN,data available,NaN,NaN,NaN,NaN


In [28]:
%%time
#### TEST without paralel computing:

all_inscriptions = []
for num in range(1,200): 
  currently_parsed = get_inscription_data(num)
  all_inscriptions.append(currently_parsed)

CPU times: user 4.97 s, sys: 404 ms, total: 5.37 s
Wall time: 2min 15s


In [51]:
%%time

### TEST with paralel computing
###to make N requests in paralel, we first have to generate a range of ranges: [1,2,3], [4,5,6], [7,8,9]
all_inscriptions = []
for num in range(1,200, 100): 
  actual_nums = list(range(num, num+100))
  with ThreadPoolExecutor(max_workers=100) as pool:
    currently_parsed = list(pool.map(get_inscription_data,actual_nums))
  all_inscriptions.extend(currently_parsed)

CPU times: user 5.27 s, sys: 497 ms, total: 5.77 s
Wall time: 17.9 s


ok, the testing clearly demonstrate that using 100 workers in paralel is about 10 times faster. Let's scale it up for the whole dataset

In [69]:
%%time
### main run of the function

all_inscriptions = []
for num in range(1,90000, 200): 
    actual_nums = list(range(num, num+200))
    with ThreadPoolExecutor(max_workers=300) as pool:
        currently_parsed = list(pool.map(get_inscription_data,actual_nums))
    all_inscriptions.extend(currently_parsed)

CPU times: user 41min 43s, sys: 3min 9s, total: 44min 53s
Wall time: 1h 24min 55s


In [70]:
all_inscriptions_filtered = []
for ins in all_inscriptions:
    try: all_inscriptions_filtered.append(ins[0])
    except: pass #[ins[0] for ins in all_inscriptions if ins != None
len(all_inscriptions_filtered) 

81476

In [71]:
inscriptions_data_df = pd.DataFrame(all_inscriptions_filtered)

In [72]:
# check missing numbers
number_set = [n for n in range(1, len(inscriptions_data_df))]
ins_ns = [int(ins.partition("HD")[2]) for ins in inscriptions_data_df["id"].tolist()]
set(ins_ns) ^ set(number_set)

{485,
 526,
 719,
 1115,
 1535,
 1797,
 2799,
 2901,
 3038,
 3044,
 3084,
 3189,
 3243,
 3282,
 3610,
 3924,
 4200,
 4223,
 4570,
 4693,
 4787,
 4902,
 5083,
 5322,
 5325,
 5328,
 5554,
 5622,
 5715,
 5872,
 5877,
 6342,
 6393,
 6424,
 6554,
 6572,
 6783,
 6852,
 7836,
 7930,
 7947,
 8145,
 8154,
 8161,
 8251,
 8263,
 8303,
 8437,
 8561,
 8576,
 8740,
 8789,
 8886,
 9084,
 9087,
 9090,
 9093,
 9096,
 9102,
 9141,
 9263,
 9293,
 9449,
 9529,
 9602,
 9682,
 9705,
 9736,
 9739,
 9817,
 9993,
 10028,
 10065,
 10068,
 10093,
 10132,
 10317,
 10320,
 10329,
 10581,
 10714,
 10717,
 10735,
 10769,
 10828,
 10874,
 10912,
 11149,
 11167,
 11276,
 11284,
 11327,
 11333,
 11336,
 11339,
 11548,
 11658,
 11880,
 12010,
 12077,
 12106,
 12124,
 12127,
 12291,
 12296,
 12318,
 12333,
 12348,
 12488,
 12572,
 12993,
 13413,
 13513,
 13709,
 13915,
 14297,
 14318,
 14386,
 14421,
 14887,
 14925,
 15069,
 15284,
 15359,
 15360,
 15496,
 15499,
 15502,
 15693,
 15694,
 15699,
 15700,
 15932,
 15935,
 1

In [73]:
inscriptions_data_df.head(5)

,responsible_individual,type_of_inscription,letter_size,not_after,literature,work_status,height,diplomatic_text,people,depth,...,not_before,findspot,year_of_find,present_location,external_image_uris,religion,fotos,geography,social_economic_legal_history,military
0,Feraudi,epitaph,3.2-2 cm,0130,"AE 1983, 0192.; M. Annecchino, Puteoli 4/5, 19...",provisional,33 cm,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,"[{'name': 'Noniae P.f. Optatae', 'cognomen': '...",2.7 cm,...,0071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Feraudi,epitaph,4 cm,0200,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",no image,28 cm,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,"[{'nomen': 'Sextius', 'praenomen': 'C.', 'pers...",NaN,...,0051,"Via Nomentana, S. Alessandro, Kirche",1937,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Feraudi,honorific inscription,4.5-3 cm,0170,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",provisional,(37) cm,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,"[{'nomen': 'Mummius+', 'gender': 'male', 'prae...",(12) cm,...,0131,NaN,before 1975,"Sevilla, Privatbesitz",NaN,NaN,NaN,NaN,NaN,NaN
3,Gräf,votive inscription,2.5 cm,0200,"AE 1983, 0533. (B); A.U. Stylow, Gerión 1, 198...",checked with photo,(39) cm,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,[{'name': '[---?]AV(?)S(?)[---]L(?)L(?)A M. Po...,18 cm,...,0151,NaN,before 1979,"Carcabuey, Grupo Escolar",[http://cil-old.bbaw.de/test06/bilder/datenban...,names of pagan deities,NaN,NaN,NaN,NaN
4,Feraudi,epitaph,NaN,0200,"AE 1983, 0078. (B); A. Ferrua, RAL 36, 1981, 1...",no image,NaN,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,"[{'person_id': '1', 'name': '[---]l. Successus...",NaN,...,0001,Via Cupa (ehem. Vigna Nardi),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
inscriptions_data_df = inscriptions_data_df[inscriptions_data_df["id"].notnull()]
len(inscriptions_data_df)

81476

# Upload the data to sciencedata.dk shared folder

In [75]:
sddk.write_file(sddk_url + "SDAM_data/EDH/EDH_onebyone_2020-09-14.json", data=inscriptions_data_df.to_json())

NameError: name 'sddk_url' is not defined